In [1]:
import logging

logger = logging.getLogger('Log.Parser1')
logger.setLevel(logging.DEBUG)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s',datefmt='%m-%d %H:%M:%S')
formatter = logging.Formatter('%(asctime)s - %(message)s',datefmt='%m-%d %H:%M:%S')
fh1 = logging.FileHandler(filename="nb_mytry.py.log", mode='a')
console = logging.StreamHandler()
console.setFormatter(formatter)
console.setLevel(logging.INFO)
fh1.setLevel(logging.DEBUG)
fh1.setFormatter(formatter)
logger.addHandler(console)
logger.addHandler(fh1)



In [2]:
def log(*args):
    logger1 = logging.getLogger('Log.Parser1')
    logger1.info(' '.join([str(arg) for arg in args]))

In [ ]:
log("st")
a = 10
log("st",a,3,4)


In [4]:
import numpy
import pandas as pd
df = pd.read_csv('db/2892_database_1.csv',encoding='utf-16',index_col='日期')
df.tail()

df['永豐金-竹北']
#df.index = pd.to_datetime(date.strftime('%Y %m %d ') + pd.Series(df.index))
#df.head()

日期
2020-12-1      6
2020-12-2      8
2020-12-3     12
2020-12-4     12
2020-12-7     26
            ... 
2021-4-7     165
2021-4-8     155
2021-4-9     144
2021-4-12    131
2021-4-13    125
Name: 永豐金-竹北, Length: 85, dtype: int64

In [10]:
import pandas as pd
import datetime
date = datetime.date(2018,1,3)
#df.index = df['日期']
df.index = pd.to_datetime(pd.Series(df.index))
#df = df.reset_index(df['日期'])
df.tail()
df.to_pickle('db/2892.pkl')

In [13]:
import pandas as pd
col = '合庫'
df1 = pd.read_pickle('db/2892.pkl')
print(df1.head())
close_  = df1['收盤']
close_.to_csv('收盤.csv',encoding='utf-16')


               收盤    漲跌     開盤     最高     最低  合庫  合庫-台中  合庫-台南  合庫-高雄  合庫-嘉義  \
日期                                                                             
2020-12-01  21.40  0.10  21.35  21.40  21.20   2      2     -3      4      3   
2020-12-02  21.35  0.05  21.25  21.35  21.20  35     17      7      5     10   
2020-12-03  21.25  0.10  21.25  21.30  21.20  52      8     26      7     23   
2020-12-04  21.35  0.10  21.30  21.40  21.25  54     19     37      8     23   
2020-12-07  21.30  0.05  21.40  21.45  21.15  64     49     75     13     27   

            ...  聯邦商銀  凱基-員林  元大-忠孝鼎富  德信-和平  石橋  國票-台東  台灣企銀-三民  光和-和美  \
日期          ...                                                           
2020-12-01  ...     0      0        0      0   0      0        0      0   
2020-12-02  ...     0      0        0      0   0      0        0      0   
2020-12-03  ...     0      0        0      0   0      0        0      0   
2020-12-04  ...     0      0        0      0   0      0        0

In [33]:
#print(df1.head())
column = ['name','sum','overbuy','day','win','count']
def stragy_sell(overbuy,d):
    #overbuy = 30
    startcal = 0
    max = 0
    total = 0

    #發現訊號那天開始算d天的收盤價 - 發現訊號那天開始的隔天收盤價買進 
    profit5 =  close_.shift(d*-1) - close_.shift(-1)
    maxlist = pd.DataFrame(columns = column)
    for col in df1.columns:
        if col == '最低':
            startcal = 1
            continue
        if startcal:        
            d1 = pd.Series(df1[col])
            d2 = d1.diff()
            #d2 = d2.dropna()
            k = profit5[d2<overbuy]
            k = k-4 #手續費 滑價
            k = k.dropna()
            p = k[k > 0]
            n = k[k <=0]
            t = k.sum()
            #t =t-(len(k) * 4)
            if t > 0:
                max = t
                #if len(k) >=10 and (len(p)/len(k))>=0.1:
                    #datac.append(col)
                datac=[[col,t,overbuy,d,round(len(p)/len(k),2),len(k)]]

                #datac=[[1,1,1,1,3,1]]
                d1 = pd.DataFrame(datac, columns=column)
                maxlist=maxlist.append(d1)
                #print('------------maxlist---------')
                #print(maxlist.head())
                
                ss = '{:<10},sum:{:},overbuy :{:}, day:{:},win:[{:.2f}%],count:{:}'.format(col,t,overbuy,d,len(p)/len(k),len(k))
                    
                #log(col , 'sum:',t , ',win:{:}-[{:.2f}%]'.format(len(p),len(p)/len(k)),',lose:{:}-[{:.2f}%]'.format(len(n),len(n)/len(k)),',count :' , len(k))
                #log(k)
                #print('n >> ' , len(n))
                #print('count>>' , len(k))
                #print(col , ">>>" , t)
                total += t
    return total,maxlist

In [35]:

#print(df1.head())
column = ['name','sum','overbuy','day','win','count']
def stragy(overbuy,d,sb):
    #overbuy = 30
    startcal = 0
    max = 0
    total = 0

    #發現訊號那天開始算d天的收盤價 - 發現訊號那天開始的隔天收盤價買進 
    profit5 =  close_.shift(d*-1) - close_.shift(-1)
    maxlist = pd.DataFrame(columns = column)
    for col in df1.columns:
        if col == '最低':
            startcal = 1
            continue
        if startcal:        
            d1 = pd.Series(df1[col])
            d2 = d1.diff()
            #d2 = d2.dropna()
            k = profit5[d2>overbuy]
            k = k-sb #手續費 滑價
            k = k.dropna()
            p = k[k > 0]
            n = k[k <=0]
            t = k.sum()
            #t =t-(len(k) * 4)
            if t > 0:
                max = t
                #if len(k) >=10 and (len(p)/len(k))>=0.1:
                    #datac.append(col)
                datac=[[col,t,overbuy,d,round(len(p)/len(k),2),len(k)]]

                #datac=[[1,1,1,1,3,1]]
                d1 = pd.DataFrame(datac, columns=column)
                maxlist=maxlist.append(d1)
                #print('------------maxlist---------')
                #print(maxlist.head())
                
                ss = '{:<10},sum:{:},overbuy :{:}, day:{:},win:[{:.2f}%],count:{:}'.format(col,t,overbuy,d,len(p)/len(k),len(k))
                    
                #log(col , 'sum:',t , ',win:{:}-[{:.2f}%]'.format(len(p),len(p)/len(k)),',lose:{:}-[{:.2f}%]'.format(len(n),len(n)/len(k)),',count :' , len(k))
                #log(k)
                #print('n >> ' , len(n))
                #print('count>>' , len(k))
                #print(col , ">>>" , t)
                total += t
    return total,maxlist



列出查詢出來的條件的詳細進場日期

In [7]:
close_  = df1['收盤']
a = close_.index[0].strftime('%Y-%m-%d')
a
#a = close_.index.astype(str)
#a
#print(a[0])


'2021-01-04'

In [29]:
close_  = df1['收盤']
d = 7
overbuy = 10
profit5 = close_.shift((d)*-1) - close_.shift(-1)
d1 = pd.Series(df1[name])
d2 = d1.diff()
#print(d2)

k = profit5[d2>overbuy]
print(k)
k = k-0.4 #手續費 滑價

#p = k[k > 0]

#p
#startdate = p.first('1D').index[0]

日期
2020-12-02   -0.40
2020-12-03   -0.70
2020-12-09    0.15
2020-12-10   -0.05
2020-12-15   -0.05
2020-12-18    0.10
2020-12-29    0.15
2021-01-05    0.30
2021-01-06   -0.20
2021-01-08   -0.60
2021-01-13   -0.90
2021-01-15   -0.50
2021-01-20   -0.30
2021-01-22   -0.30
2021-01-26    0.00
2021-01-28    0.60
2021-01-29    0.55
2021-02-01    0.50
2021-02-04    0.50
2021-02-19    0.45
2021-02-22   -0.05
2021-02-25    0.65
2021-02-26    0.45
2021-03-08    0.40
2021-03-10   -0.10
2021-03-11    0.10
2021-03-17    0.10
2021-03-19    0.40
2021-03-22    0.35
2021-03-25    0.15
2021-03-26   -0.10
2021-03-30   -0.05
2021-04-01     NaN
2021-04-06     NaN
2021-04-09     NaN
Name: 收盤, dtype: float64


In [30]:

overbuy = -10
d = 7
name = '合庫'#'永豐金-竹北'
#total,m,p = stragy(overbuy,d)

#profit5 =  close_.shift(d*-1) + close_.shift(-1)
close_  = df1['收盤']
#print(close_.head(30))
close_.to_csv(name+'_close.csv',encoding='utf-16')
#發現訊號那天開始算d天 - 發現訊號那天開始的隔天買進 
profit5 = close_.shift((d)*-1) - close_.shift(-1)
#print(profit5.head(30))
profit5.to_csv(name+'_profit5.csv',encoding='utf-16')

d1 = pd.Series(df1[name])
#print(d1)
d2 = d1.diff()
d2.to_csv(name+'overbuy.csv',encoding='utf-16')
#d2 = d2.dropna()
#print(d2)
k = profit5[d2<overbuy]
#print(k)
k = k-0.2 #手續費 滑價
#print(k)
p = k[k > 0]

#取得訊號發生第一筆
startdate = p.first('1D').index[0]
startdate = startdate.strftime('%Y-%m-%d')
print('start=',p)
clos_startbydate = close_
clos_startbydate = clos_startbydate[startdate:]
#從隔天價格開始買, 所以剔除訊號發生那天
clos_startbydate = clos_startbydate[1:]
print (clos_startbydate.head())
earn_percentage = (clos_startbydate - clos_startbydate[0]) *100
earn_percentage = (earn_percentage/clos_startbydate[0])
print(earn_percentage)
#s = close_[p.first('1D').index:]
#print(s)
n = k[k <=0]
t = k.sum()
print(name)
print(p)
p.to_csv(name+'_buy.csv',encoding='utf-16')



start= 日期
2021-02-02    0.20
2021-03-02    0.15
2021-03-04    0.65
Name: 收盤, dtype: float64
日期
2021-02-03    20.40
2021-02-04    20.40
2021-02-05    20.60
2021-02-17    20.80
2021-02-18    20.85
Name: 收盤, dtype: float64
日期
2021-02-03    0.000000
2021-02-04    0.000000
2021-02-05    0.980392
2021-02-17    1.960784
2021-02-18    2.205882
2021-02-19    2.450980
2021-02-22    1.960784
2021-02-23    3.431373
2021-02-24    3.431373
2021-02-25    4.656863
2021-02-26    2.205882
2021-03-02    3.676471
2021-03-03    4.166667
2021-03-04    3.186275
2021-03-05    3.431373
2021-03-08    3.431373
2021-03-09    5.392157
2021-03-10    5.882353
2021-03-11    5.882353
2021-03-12    6.617647
2021-03-15    7.598039
2021-03-16    8.578431
2021-03-17    7.352941
2021-03-18    7.598039
2021-03-19    5.392157
2021-03-22    7.107843
2021-03-23    7.107843
2021-03-24    7.352941
2021-03-25    8.088235
2021-03-26    8.088235
2021-03-29    8.823529
2021-03-30    9.068627
2021-03-31    8.823529
2021-04-01    8.08

開始模擬

In [36]:
total ,maxlist= stragy_sell(-200,4)
maxlist

,name,sum,overbuy,day,win,count
0,合庫,5.0,-200,4,1.0,1
0,日盛-頭份,23.5,-200,4,0.5,2
0,宏遠,19.0,-200,4,0.4,5
0,美林,45.5,-200,4,1.0,2
0,新加坡商瑞銀,2.5,-200,4,0.5,2
0,兆豐-台中,1.5,-200,4,1.0,1
0,國泰,10.0,-200,4,1.0,1
0,國泰-館前,9.0,-200,4,0.5,2
0,凱基-復興,22.5,-200,4,1.0,1
0,永豐金,1.5,-200,4,1.0,1


In [36]:

max = 0
ss = ''
maxtotal = pd.DataFrame(columns = column)
for s in range(30,300,10):
    for d in range(2,30):
        log('overbuy : ' + str(s)+' day :' + str(d))
        total ,maxlist= stragy(s,d,0.4)
        if total > max:
            ss = 'overbuy : ' + str(s)+' day :' + str(d)
            max = total
        log('total:' + str(total)+'\n')
        maxtotal = maxtotal.append(maxlist)
#for i in ran
maxtotal = maxtotal.reset_index(drop=True)
print(maxtotal.head())
maxtotal.to_csv('maxtotal_buy.csv',encoding='utf-16')
#log('----final-------')
#log(ss)
#log('----single best-------')
#for i in range(0,len(maxlist)):
    #log(maxlist[i])

13 - overbuy : 210 day :7
04-14 15:12:15 - total:3.750000000000016

04-14 15:12:15 - overbuy : 210 day :8
04-14 15:12:16 - total:4.700000000000018

04-14 15:12:16 - overbuy : 210 day :9
04-14 15:12:18 - total:7.100000000000013

04-14 15:12:18 - overbuy : 210 day :10
04-14 15:12:19 - total:10.4

04-14 15:12:19 - overbuy : 210 day :11
04-14 15:12:21 - total:14.149999999999993

04-14 15:12:21 - overbuy : 210 day :12
04-14 15:12:22 - total:16.89999999999999

04-14 15:12:22 - overbuy : 210 day :13
04-14 15:12:24 - total:19.149999999999984

04-14 15:12:24 - overbuy : 210 day :14
04-14 15:12:25 - total:26.500000000000007

04-14 15:12:25 - overbuy : 210 day :15
04-14 15:12:27 - total:29.75000000000001

04-14 15:12:27 - overbuy : 210 day :16
04-14 15:12:29 - total:36.50000000000004

04-14 15:12:29 - overbuy : 210 day :17
04-14 15:12:30 - total:38.55000000000004

04-14 15:12:30 - overbuy : 210 day :18
04-14 15:12:32 - total:39.500000000000036

04-14 15:12:32 - overbuy : 210 day :19
04-14 15:12:3